In [2]:
import os
import cv2
import xml.etree.ElementTree as ET
from skimage.feature import hog
from skimage.color import rgb2gray
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
import numpy as np

In [3]:
def extract_hog_features(image):
    gray = rgb2gray(image)
    features = hog(
        gray,
        orientations=9,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm='L2-Hys',
        visualize=False
    )
    return features

In [4]:
# Paths
image_folder = '/home/asit-patel/Object_detection/main_working/bottle/final_images'
annotation_folder = '/home/asit-patel/Object_detection/main_working/bottle/final_xml'

# Containers
X = []
y = []

# Resize all objects to same size
target_size = (64, 128)

# HOG function
def extract_hog_features(image):
    gray = rgb2gray(image)
    features = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                   cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
    return features

# Iterate through all annotations
for filename in os.listdir(annotation_folder):
    if not filename.endswith(".xml"):
        continue
    filepath = os.path.join(annotation_folder, filename)
    tree = ET.parse(filepath)
    root = tree.getroot()

    image_filename = root.find("filename").text
    image_path = os.path.join(image_folder, image_filename)
    image = cv2.imread(image_path)
    if image is None:
        continue

    for obj in root.findall("object"):
        label = obj.find("name").text.strip().lower()
        bbox = obj.find("bndbox")
        xmin = int(float(bbox.find("xmin").text))
        ymin = int(float(bbox.find("ymin").text))
        xmax = int(float(bbox.find("xmax").text))
        ymax = int(float(bbox.find("ymax").text))

        cropped = image[ymin:ymax, xmin:xmax]
        resized = cv2.resize(cropped, target_size)
        features = extract_hog_features(resized)
        X.append(features)
        y.append(label)

FileNotFoundError: [Errno 2] No such file or directory: '/home/asit-patel/Object_detection/main_working/bottle/final_xml'

In [7]:
print(set(y))

{'book', 'background', 'bottle'}


In [8]:
# Convert to numpy
X = np.array(X)
y = np.array(y)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train SVM (multi-class)
model = SVC(kernel='linear', probability=True)
model.fit(X_train, y_train)

# Evaluate
print("Classification Report:")
print(classification_report(y_test, model.predict(X_test)))

Classification Report:
              precision    recall  f1-score   support

  background       0.96      1.00      0.98        66
        book       0.83      0.77      0.80        13
      bottle       0.93      0.81      0.87        16

    accuracy                           0.94        95
   macro avg       0.91      0.86      0.88        95
weighted avg       0.93      0.94      0.93        95



In [9]:
import joblib
joblib.dump(model, 'bottle_classifier.pkl')

['bottle_classifier.pkl']

In [10]:
len(y)

474